# Prep dataset

In [1]:
import pandas as pd
import numpy as np
import os
import scipy
import scipy.stats

from google.colab import drive
drive.mount('/content/drive')
data_foloder = '/content/drive/My Drive/simulation/Gradient_shuffle/'
result = '/content/drive/My Drive/simulation/Output_simul/BERT/sentence/'

baseline = pd.read_csv(data_foloder + 'simulation_HV_baseline_vb_response_deid_v3.csv')
incoh10 = pd.read_csv(data_foloder + 'simulation_HV_incoh_vb_response_deid_10v3.csv')
incoh20 = pd.read_csv(data_foloder + 'simulation_HV_incoh_vb_response_deid_20v3.csv')
incoh50 = pd.read_csv(data_foloder + 'simulation_HV_incoh_vb_response_deid_50v3.csv')
ineff10 = pd.read_csv(data_foloder + 'simulation_HV_ineff_vb_response_deid_10v3.csv')
ineff20 = pd.read_csv(data_foloder + 'simulation_HV_ineff_vb_response_deid_20v3.csv')
ineff50 = pd.read_csv(data_foloder + 'simulation_HV_ineff_vb_response_deid_50v3.csv')

baseline.head()

Mounted at /content/drive


,grid,content,n_words
0,10455,"I'm a young man , an en an en- an engineer by ...",421
1,11689,Sure . I'm thirty three years old . My name is...,159
2,12376,Alright . um I live in not especially cool Spr...,468
3,12630,um So I'm currently twenty-nine . I was born a...,966
4,13493,Mhm . I'm a thirty five year old man who uh um...,134


In [2]:
os.getcwd()

'/content'

# Install lib and dependencies

In [3]:
# install sentence-transformers from hugging face
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.6 MB/s 
     |████████████████████████████████| 5.5 MB 28.8 MB/s 
     |████████████████████████████████| 1.3 MB 55.1 MB/s 
     |████████████████████████████████| 182 kB 58.7 MB/s 
     |████████████████████████████████| 7.6 MB 39.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=023521ed55d95c03fd6587c3abfb1c0b197841ab9726360a7529954e2f309722
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [4]:
# Utilize BERT
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-roberta-large')
type(model) #if worked well, should print: sentence_transformers.SentenceTransformer.SentenceTransformer

Downloading:   0%|          | 0.00/748 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

sentence_transformers.SentenceTransformer.SentenceTransformer

# Contextualized sentence embeddings

In [15]:
def get_sent_emb(input):
  sentence = input.split('.')[:-1]
  embedding = model.encode(sentence, convert_to_tensor=True)
  return embedding

# call the function 
# apply it to the dataframe
dfs = [baseline, incoh10, incoh20, incoh50, ineff10, ineff20, ineff50]
temp = -1
for df in dfs:
  temp += 1
  df["bert_sent_emb"] = ''
  df['bert_sent_emb'] = df['content'].apply(lambda x: get_sent_emb(x))
  df.to_csv(result + str(temp) + '_sent.csv')
df.head()

,grid,content,n_words,bert_sent_emb
0,10455,"I'm a young man , an en an en- an engineer by ...",421,"[[tensor(2.2570), tensor(0.1405), tensor(0.423..."
1,11689,We have been using that opportunity to do more...,159,"[[tensor(-0.8558), tensor(0.5730), tensor(0.25..."
2,12376,Alright . um I live in not especially cool Spr...,468,"[[tensor(-0.4359), tensor(-0.2084), tensor(-0...."
3,12630,"My is things are fantastic . No , I mean My uh...",966,"[[tensor(-1.4369), tensor(-0.4281), tensor(-0...."
4,13493,Mhm . I still get to play . And my stock inves...,134,"[[tensor(-0.6955), tensor(-0.3440), tensor(-0...."


In [16]:
df.columns

Index(['grid', 'content', 'n_words', 'bert_sent_emb'], dtype='object')

# Stats and similarities functions

In [17]:
# stats ignoring nan, apply to all LMs 
from numpy import nanmedian

import scipy
def iqr(x):
  return scipy.stats.iqr(np.array(x), nan_policy='omit')

from numpy import nanquantile
def q5(x):
    return np.nanquantile(np.array(x), 0.05)

def q95(x):
    return np.nanquantile(np.array(x), 0.95)

In [18]:
# cosine_similarity, apply to all LMs
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# FOC and SOC

In [19]:
cs=['foc', 'soc']
stats = ['_median', '_iqr', '_q5', '_q95']
for df in dfs:
    # create new empty columns
    for c in cs:
        for stat in stats:
            cur = 'bert_' + c + stat
            df[cur] = ''
df.head()

,grid,content,n_words,bert_sent_emb,bert_foc_median,bert_foc_iqr,bert_foc_q5,bert_foc_q95,bert_soc_median,bert_soc_iqr,bert_soc_q5,bert_soc_q95
0,10455,"I'm a young man , an en an en- an engineer by ...",421,"[[tensor(2.2570), tensor(0.1405), tensor(0.423...",,,,,,,,
1,11689,We have been using that opportunity to do more...,159,"[[tensor(-0.8558), tensor(0.5730), tensor(0.25...",,,,,,,,
2,12376,Alright . um I live in not especially cool Spr...,468,"[[tensor(-0.4359), tensor(-0.2084), tensor(-0....",,,,,,,,
3,12630,"My is things are fantastic . No , I mean My uh...",966,"[[tensor(-1.4369), tensor(-0.4281), tensor(-0....",,,,,,,,
4,13493,Mhm . I still get to play . And my stock inves...,134,"[[tensor(-0.6955), tensor(-0.3440), tensor(-0....",,,,,,,,


In [20]:
temp_file = -1 # keep track of file names
for df in dfs:
    temp_file += 1
    df['bert_foc_similarity'] = ''
    df['bert_soc_similarity'] = ''
    # loop over each response
    for i in df.index:
        temp_foc = [] # for each individual, store the list of cos similarity
        temp_soc = []
        # calculate average similarity for sentence pairs, either adjacent or with one intervening
        for idx, sent in enumerate(df['bert_sent_emb'][i]):
            try:
                temp_foc.append(cosine_similarity(sent, df['bert_sent_emb'][i][idx+1])) # get a list of similarities for that response
                
                df['bert_foc_similarity'][i] = temp_foc # record intermediate similarities 
                
                df['bert_foc_median'][i] = np.nanmedian(temp_foc) # add more stats here
                df['bert_foc_iqr'][i] = iqr(temp_foc)
                df['bert_foc_q5'][i] = q5(temp_foc)
                df['bert_foc_q95'][i] = q95(temp_foc)

                temp_soc.append(cosine_similarity(sent, df['bert_sent_emb'][i][idx+2]))

                df['bert_soc_similarity'][i] = temp_soc # record intermediate similarities 

                df['bert_soc_median'][i] = np.nanmedian(temp_soc)
                df['bert_soc_iqr'][i] = iqr(temp_soc)
                df['bert_soc_q5'][i] = q5(temp_soc)
                df['bert_soc_q95'][i] = q95(temp_soc)

            except IndexError:
                continue
    df.to_csv(result + str(temp_file) + '_sent.csv')
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: Settin

,grid,content,n_words,bert_sent_emb,bert_foc_median,bert_foc_iqr,bert_foc_q5,bert_foc_q95,bert_soc_median,bert_soc_iqr,bert_soc_q5,bert_soc_q95,bert_foc_similarity,bert_soc_similarity
0,10455,"I'm a young man , an en an en- an engineer by ...",421,"[[tensor(2.2570), tensor(0.1405), tensor(0.423...",0.206066,0.104518,0.119734,0.358415,0.139063,0.146933,-0.000956,0.345452,"[0.08307609, 0.14944445, 0.20996395, 0.2249982...","[0.07057538, 0.1555113, 0.13906263, 0.05380263..."
1,11689,We have been using that opportunity to do more...,159,"[[tensor(-0.8558), tensor(0.5730), tensor(0.25...",0.189839,0.11959,0.026358,0.256293,0.227235,0.218465,-0.021945,0.413549,"[0.08995072, 0.14319871, 0.045608412, 0.219003...","[0.09434952, -0.09893811, 0.07215834, 0.079532..."
2,12376,Alright . um I live in not especially cool Spr...,468,"[[tensor(-0.4359), tensor(-0.2084), tensor(-0....",0.123315,0.264068,-0.03481,0.625657,0.192952,0.250752,0.016645,0.417292,"[-0.0025921687, 0.06616052, 0.43268007, 0.0577...","[0.37668362, 0.2500623, 0.21057622, 0.37759405..."
3,12630,"My is things are fantastic . No , I mean My uh...",966,"[[tensor(-1.4369), tensor(-0.4281), tensor(-0....",0.152373,0.189017,-0.048281,0.514207,0.134735,0.222171,-0.056714,0.486469,"[0.15253223, -0.048458852, 0.14621271, 0.14502...","[0.12442489, -0.05728216, 0.0909018, 0.2215588..."
4,13493,Mhm . I still get to play . And my stock inves...,134,"[[tensor(-0.6955), tensor(-0.3440), tensor(-0....",0.226927,0.11738,0.126698,0.332373,0.200609,0.319422,-0.026909,0.435249,"[0.1697056, 0.11594563, 0.3436946, 0.22692677,...","[-0.044122603, 0.0706351, 0.45371968, 0.3305834]"


# TLC merge

In [ ]:
tlc = pd.read_csv('/Users/yancong/Desktop/4 clinical/00 Project Files/crossdx_clin.csv', index_col=0)
tlc = tlc[['grid', 'SSDvHC', 'group', 'tlc_01povspeech', 'tlc_02povcontent',	'tlc_03pressure',	'tlc_04distract',
	'tlc_05tangent', 'tlc_06derail', 'tlc_07incoh',	'tlc_08illogic',	'tlc_09clang',	'tlc_10neologism',
    	'tlc_11wordapprox',	'tlc_12circum',	'tlc_13lossgoal',	'tlc_14persev',	'tlc_15echo',	'tlc_16block',
        	'tlc_17stilt',	'tlc_18selfref', 'tlc_3f_inefficient',	'tlc_3f_incoherent',	'tlc_3f_impexpress']]
tlc.head()

In [ ]:
# only keep the processed stats columns
temp = -1
for df in dfs:
    temp += 1
    df.drop(['content', 'n_words', 'bert_embed'], axis = 1, inplace=True)
    df['grid'] = df['grid'].astype(str)
    tlc['grid'] = tlc['grid'].astype(str)
    df = df.merge(tlc, on=['grid'])
    df.to_csv(result + str(temp) + '_GT.csv')
df.head()

In [ ]:
df.columns